##Install Pre req from Kaggle

Download the required datasets from the Kaggle repository.

In [ ]:
!pip install kaggle

In [ ]:
!mkdir ~/.kaggle

In [ ]:
cd /root/.kaggle

In [ ]:
!pwd
!ls

Upload the json file for Kaggle authorization

In [ ]:
from google.colab import files
files.upload()

In [12]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
cd /content

In [ ]:
!kaggle competitions download -c nlp-getting-started

#Import required libs

In [20]:
import numpy as np
import pandas as pd


import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import json


pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.width = None
pd.options.display.max_colwidth = None

#Loading the data

In [21]:
Tweets = pd.read_csv("/content/train.csv", encoding="utf-8")

#Helper functions

Some helper functions for preprocessing the tweets are defined here

Load the dictionaries from the GitHub repositiory to process contracted words and words with repeated characters.

**Contracted words**




Eg  I've  = I have

**Repeated characters words**




Eg  goooaaalll  = goal

coooool     = cool
    

In [22]:
with open('contractions_dict.json', 'r') as fp:
    contractions_dict = json.load(fp)
with open('repetitions_dict.json', 'r') as fp:
    repetitions_dict = json.load(fp)
print('Contracted Words = ', len(contractions_dict), 'Repetition Letter Words = ',len(repetitions_dict))

Contracted Words =  107 Repetition Letter Words =  13


Helper functions for basic text processing, tweet processing, extracting features and tokenization

In [23]:
# To extract URLs as a seperate feature
def ExtractURL(text):
  match = re.findall('https?://\S+|www\.\S+', text)
  if match == []:
    url = np.nan
  else:
    url = match
  return url

# To extract Hashtags as a seperate feature
def Extracthashtag(text):
  match = re.findall('#\S+', text)
  if match == []:
    hashtag = np.nan
  else:
    hashtag = match
    for idx, i in enumerate(hashtag):
      hashtag[idx] = re.sub('[%s]' % re.escape(string.punctuation), '', i)
  return hashtag

# To perfrom tweets specific text processing
def FixCommonTweetErrors(text, remove_hashtag):
  # To expand contracted words
  def expand_contractions(text):
    for keys,vals in contractions_dict.items():
      text = re.sub(keys, vals, text)
    return text
  # To contract words with repeated letters
  def contract_repetitions(text):
    for keys,vals in repetitions_dict.items():
      text = re.sub(keys, vals, text)
    return text
  # To remove hashtaged words from text if necessary
  if remove_hashtag:
    text = re.sub('#\S+', '', text)

  text = contract_repetitions(text)
  text = expand_contractions(text)
  return text   
    
     
def BasicTextProcessing(text, remove_url, lower_text_case,
                        remove_nos, remove_punctutation,
                        ):
  # Removing line breaks, html tags, certain unicode symbols
  # To remove url, nos and punctutation and convert to lower case if necessary
  if remove_url:
    text = re.sub('https?://\S+|www\.\S+', '', text)
  if lower_text_case:
    text = text.lower()
  text = re.sub('\\n', '', text)
  text = re.sub('ûª', "'", text)
  text = re.sub('&amp;', 'and', text)
  text = re.sub('&lt;', '', text)
  text = re.sub('&gt;', '', text)
  if remove_nos:
    text = re.sub('\d+', '', text)
  if remove_punctutation:
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)

  return text

#   text = re.sub('\[.*?\]', '', text)
#   text = re.sub('<.*?>+', '', text)
#   text = re.sub('\w*\d\w*', '', text)
  
tokenizer=nltk.tokenize.RegexpTokenizer(r'\w+')

# Downloading Stopwords
def LoadStopwords():
  nltk.download('stopwords')
  stopwords.words('english')
  # len(stopwords.words('english'))

# To remove stopwords if necessary
def RemoveStopwords(text):
    words = [w for w in text if w not in stopwords.words('english')]
    return words 

# To combine tokenized words if necessary
def CombineText(list_of_text):
    combined_text = ' '.join(list_of_text)
    return combined_text

#Pipeline

Pipleline to perfrom all preprocessing functions.

Non - required for preprocessing can be made false

**Default values**

remove_url=True, extract_url=False, remove_punctutation=True, extract_hashtag=False, 
                  lower_text_case=True, remove_nos=True, fix_common_tweet_errors=False,
                  remove_hashtag=False, remove_stopwords=False, combine_text=False

In [24]:
def CleanTextData(Data, remove_url=True, extract_url=False, remove_punctutation=True, extract_hashtag=False, 
                  lower_text_case=True, remove_nos=True, fix_common_tweet_errors=False,
                  remove_hashtag=False, remove_stopwords=False, combine_text=False
                  ):
  Data = Data.assign(text_pre_pro = lambda x: (x['text'].apply(lambda x:BasicTextProcessing(x, remove_url, lower_text_case,
                        remove_nos, remove_punctutation))))

  if fix_common_tweet_errors:
    Data['text_pre_pro'] = Data['text_pre_pro'].apply(lambda x:FixCommonTweetErrors(x, remove_hashtag)) 

  if extract_hashtag:
      Data = Data.assign(hashtag=lambda x: (x['text'].apply(lambda x:Extracthashtag(x))))

  if extract_url:
    Data = Data.assign(url=lambda x: (x['text'].apply(lambda x:ExtractURL(x))))

  Data = Data.assign(tokenized_processed=lambda x: (x['text_pre_pro'].apply(lambda x:tokenizer.tokenize(x))))

  if remove_stopwords:
    Data['text_pre_pro'] = Data['text_pre_pro'].apply(lambda x:FixCommonTweetErrors(x, remove_stopwords)) 

  if combine_text:
    Data['tokenized_processed'] = Data['text_pre_pro'].apply(lambda x:FixCommonTweetErrors(x, combine_text)) 

  # Data['text'] = Data['text'].apply(lambda x : remove_stopwords(x))
  # Data['text'] = Data['text'].apply(lambda x : combine_text(x))
  return Data

#Processing the data

In [25]:
Tweets_PreProcessed = CleanTextData(Tweets, extract_hashtag=True, 
                                        fix_common_tweet_errors=True,
                                        extract_url = True,

                                        )

# Some ids of tweets which can be used to check if the prrocssing had the inteded
# effect on the tweets
# listofid = [28, 158, 220, 232, 1722]

# for i in listofid:
#   print(Tweets_PreProcessed.loc[Tweets_PreProcessed['id'] == i, ['text', 'text_pre_pro']])

Tweets_PreProcessed.head(4)

,id,keyword,location,text,target,text_pre_pro,hashtag,url,tokenized_processed
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1,our deeds are the reason of this earthquake may allah forgive us all,[earthquake],NaN,"[our, deeds, are, the, reason, of, this, earthquake, may, allah, forgive, us, all]"
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada,NaN,NaN,"[forest, fire, near, la, ronge, sask, canada]"
2,5,NaN,NaN,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,1,all residents asked to shelter in place are being notified by officers no other evacuation or shelter in place orders are expected,NaN,NaN,"[all, residents, asked, to, shelter, in, place, are, being, notified, by, officers, no, other, evacuation, or, shelter, in, place, orders, are, expected]"
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation orders in California",1,people receive wildfires evacuation orders in california,[wildfires],NaN,"[people, receive, wildfires, evacuation, orders, in, california]"


In [26]:
Tweets_PreProcessed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   7613 non-null   int64 
 1   keyword              7552 non-null   object
 2   location             5080 non-null   object
 3   text                 7613 non-null   object
 4   target               7613 non-null   int64 
 5   text_pre_pro         7613 non-null   object
 6   hashtag              1756 non-null   object
 7   url                  3971 non-null   object
 8   tokenized_processed  7613 non-null   object
dtypes: int64(2), object(7)
memory usage: 535.4+ KB


In [27]:
Tweets_PreProcessed.to_csv('train_processed.csv')

Reset the variables if necessary

In [28]:
# %reset

#Sub Program to write new values to dicts

In [29]:
contractions_dict.items()

dict_items([("ain't", 'are not'), ("'s", ' is'), ("aren't", 'are not'), ("can't", 'cannot'), ("can't've", 'cannot have'), ("'cause", 'because'), ("could've", 'could have'), ("couldn't", 'could not'), ("couldn't've", 'could not have'), ("didn't", 'did not'), ("doesn't", 'does not'), ("don't", 'do not'), ("hadn't", 'had not'), ("hadn't've", 'had not have'), ("hasn't", 'has not'), ("haven't", 'have not'), ("he'd", 'he would'), ("he'd've", 'he would have'), ("he'll", 'he will'), ("he'll've", 'he will have'), ("how'd", 'how did'), ("how'd'y", 'how do you'), ("how'll", 'how will'), ("i'd", 'i would'), ("i'd've", 'i would have'), ("i'll", 'i will'), ("i'll've", 'i will have'), ("i'm", 'i am'), ("i've", 'i have'), ("isn't", 'is not'), ("it'd", 'it would'), ("it'd've", 'it would have'), ("it'll", 'it will'), ("it'll've", 'it will have'), ("let's", 'let us'), ("ma'am", 'madam'), ("mayn't", 'may not'), ("might've", 'might have'), ("mightn't", 'might not'), ("mightn't've", 'might not have'), ("mus

Add any new values to the dicts

In [30]:
contractions_dict['CONTRACTED WORD'] = 'EXPANDED WORD'
repetitions_dict['REPEATED LETTER WORD'] = 'CONTRACTED WORD'

Save the dict and use it for further preprocessing

In [31]:
with open('contractions_dict.json', 'w') as fp:
    json.dump(contractions_dict, fp)
with open('repetitions_dict.json', 'w') as fp:
    json.dump(repetitions_dict, fp)

#Sub Program to find list of words with repeated leterrs (Eg: gooooaaalll)

Check the number of words with repeated letters in your dataset and add any new words to the dict if necessary 

In [32]:
word_corpus = []
j = 0
for tweet in Tweets_PreProcessed['tokenized_processed']:
  for word in tweet:
    if word in word_corpus:
      j = j +1
    else:
      word_corpus.append(word)
print(len(word_corpus))

17118


In [33]:
Consec_2_letter_words = []
for word in word_corpus:
  for idx, char in enumerate(word):
    if idx + 1 < len(word):
      if word[idx] == word[idx + 1]:
        Consec_2_letter_words.append(word)
print(len(Consec_2_letter_words))
print(Consec_2_letter_words)

4160
['deeds', 'allah', 'all', 'officers', 'school', 'flood', 'flooding', 'streets', 'hill', 'see', 'woods', 'happening', 'across', 'street', 'three', 'getting', 'flooded', 'gonna', 'arrived', 'summer', 'cool', 'skiing', 'week', 'bbcmtd', 'look', 'will', 'office', 'soooo', 'soooo', 'soooo', 'followers', 'better', 'setting', 'alexissanchez', 'happy', 'teammates', 'gunners', 'offices', 'voortrekker', 'voortrekker', 'kiss', 'filled', 'peeps', 'farewell', 'progressive', 'greetingsin', 'fallen', 'hood', 'buff', 'accident', 'impossible', 'nashvilletraffic', 'nashvilletraffic', 'traffic', 'caraccidentlawyer', 'speeding', 'teen', 'accidents', 'tee', 'curry', 'awareness', 'mooresville', 'mooresville', 'iredell', 'sleepjunkies', 'sleeping', 'pills', 'happen', 'cabrillo', 'hwymagellan', 'accidentwho', 'mayonnaise', 'horrible', 'finally', 'pissed', 'donnie', 'tell', 'been', 'ashville', 'crossed', 'naayf', 'chandanee', 'mma', 'rammed', 'eddy', 'willis', 'aashiqui', 'actress', 'aggarwal', 'suffield'

In [34]:
Consec_3_letter_words = []
for word in word_corpus:
  for idx, char in enumerate(word):
    if idx + 2 < len(word):
      if word[idx] == word[idx + 1] == word[idx + 2]:
        Consec_3_letter_words.append(word)
print(len(Consec_3_letter_words))
print(Consec_3_letter_words)

196
['soooo', 'soooo', 'awwww', 'awwww', 'mhmmm', 'alexshipppp', 'alexshipppp', 'avysss', 'aiii', 'baaaack', 'baaaack', 'iii', 'xdojjjj', 'xdojjjj', 'omgbethersss', 'xxx', 'grrrr', 'grrrr', 'wwwbigbaldhead', 'zzzz', 'zzzz', 'lmfaoooo', 'lmfaoooo', 'deeeznvtzzz', 'deeeznvtzzz', 'oooureli', 'mmm', 'sniiiiiiff', 'sniiiiiiff', 'sniiiiiiff', 'sniiiiiiff', 'uhhhhh', 'uhhhhh', 'uhhhhh', 'ieee', 'rokiieee', 'loveyouuuu', 'loveyouuuu', 'sooo', 'michelleellle', 'emaaalay', 'crosssectarian', 'naaa', 'kwaaaaadead', 'kwaaaaadead', 'kwaaaaadead', 'ssshhheeesshh', 'ssshhheeesshh', 'ssshhheeesshh', 'pusssssssssy', 'pusssssssssy', 'pusssssssssy', 'pusssssssssy', 'pusssssssssy', 'pusssssssssy', 'pusssssssssy', 'shoook', 'frackfreeeu', 'stiiilo', 'ahhhhh', 'ahhhhh', 'ahhhhh', 'alllivesmatter', 'ohyayyyyay', 'ohyayyyyay', 'ayyy', 'aaaaaaallll', 'aaaaaaallll', 'aaaaaaallll', 'aaaaaaallll', 'aaaaaaallll', 'aaaaaaallll', 'aaaaaaallll', 'ssssnell', 'ssssnell', 'www', 'iiii', 'iiii', 'riveeeeeer', 'riveeeeeer'

In [35]:
Consec_4_letter_words = []
for word in word_corpus:
  for idx, char in enumerate(word):
    if idx + 3 < len(word):
      if word[idx] == word[idx + 1] == word[idx + 2] == word[idx + 3]:
        Consec_4_letter_words.append(word)
print(len(Consec_4_letter_words))
print(Consec_4_letter_words)

84
['soooo', 'awwww', 'alexshipppp', 'baaaack', 'xdojjjj', 'grrrr', 'zzzz', 'lmfaoooo', 'sniiiiiiff', 'sniiiiiiff', 'sniiiiiiff', 'uhhhhh', 'uhhhhh', 'loveyouuuu', 'kwaaaaadead', 'kwaaaaadead', 'pusssssssssy', 'pusssssssssy', 'pusssssssssy', 'pusssssssssy', 'pusssssssssy', 'pusssssssssy', 'ahhhhh', 'ahhhhh', 'ohyayyyyay', 'aaaaaaallll', 'aaaaaaallll', 'aaaaaaallll', 'aaaaaaallll', 'aaaaaaallll', 'ssssnell', 'iiii', 'riveeeeeer', 'riveeeeeer', 'riveeeeeer', 'milioooo', 'roomsgrrrr', 'alllll', 'alllll', 'nowwwwww', 'nowwwwww', 'nowwwwww', 'wompppp', 'selmoooooo', 'selmoooooo', 'selmoooooo', 'errrr', 'damnnnn', 'aannnnd', 'shidddd', 'ahhhh', 'omgggg', 'aaaa', 'onnnn', 'comeeeee', 'comeeeee', 'mxaaaa', 'llll', 'blaaaaaaa', 'blaaaaaaa', 'blaaaaaaa', 'blaaaaaaa', 'ayhhhhhdjjfjrjjrdjjeks', 'ayhhhhhdjjfjrjjrdjjeks', 'mochichiiiii', 'mochichiiiii', 'sheetingaaaaaand', 'sheetingaaaaaand', 'sheetingaaaaaand', 'maaaaan', 'maaaaan', 'ruddyyyyyy', 'ruddyyyyyy', 'ruddyyyyyy', 'caaaaaall', 'caaaaaall'

In [36]:
Consec_4_wr_letter_words = []
for word in Consec_4_letter_words:
  new_word = ''
  for idx, char in enumerate(word):
    if idx + 1 < len(word):
      
      if word[idx] != word[idx + 1]:
        # print(word[idx], word[idx + 1])
        new_word = new_word + char
    else:
      # if word[idx - 1] == word[idx]:
      new_word = new_word + char
  Consec_4_wr_letter_words.append(new_word)
        
print(len(Consec_4_wr_letter_words))
print(Consec_4_wr_letter_words)

84
['so', 'aw', 'alexship', 'back', 'xdoj', 'gr', 'z', 'lmfao', 'snif', 'snif', 'snif', 'uh', 'uh', 'loveyou', 'kwadead', 'kwadead', 'pusy', 'pusy', 'pusy', 'pusy', 'pusy', 'pusy', 'ah', 'ah', 'ohyayay', 'al', 'al', 'al', 'al', 'al', 'snel', 'i', 'river', 'river', 'river', 'milio', 'romsgr', 'al', 'al', 'now', 'now', 'now', 'womp', 'selmo', 'selmo', 'selmo', 'er', 'damn', 'and', 'shid', 'ah', 'omg', 'a', 'on', 'come', 'come', 'mxa', 'l', 'bla', 'bla', 'bla', 'bla', 'ayhdjfjrjrdjeks', 'ayhdjfjrjrdjeks', 'mochichi', 'mochichi', 'shetingand', 'shetingand', 'shetingand', 'man', 'man', 'rudy', 'rudy', 'rudy', 'cal', 'cal', 'cal', 'wo', 'wo', 'wo', 'wo', 'rios', 'rios', 'nice']


In [37]:
from collections import Counter

new_words = Counter(Consec_4_wr_letter_words)

for j, i in new_words.items():
  print(j, i)

so 1
aw 1
alexship 1
back 1
xdoj 1
gr 1
z 1
lmfao 1
snif 3
uh 2
loveyou 1
kwadead 2
pusy 6
ah 3
ohyayay 1
al 7
snel 1
i 1
river 3
milio 1
romsgr 1
now 3
womp 1
selmo 3
er 1
damn 1
and 1
shid 1
omg 1
a 1
on 1
come 2
mxa 1
l 1
bla 4
ayhdjfjrjrdjeks 2
mochichi 2
shetingand 3
man 2
rudy 3
cal 3
wo 4
rios 2
nice 1
